# 📗 Sanity Check Notebook — PINN Greeks Project

Use this notebook to **verify your environment, data, baselines, and a tiny PINN training loop** end-to-end.

**What this does:**
1. Prints package versions and creates required folders
2. Imports Black–Scholes utilities (or defines fallbacks)
3. Generates a *small* synthetic dataset for quick checks
4. Plots price & Greek curves
5. Runs finite-difference and Monte Carlo baselines vs analytic
6. Trains a **tiny** PINN for a few epochs to confirm gradients & loss
7. Plots predicted surface and a rough PDE residual heatmap

> ⚠️ This is **not** for full experiments — keep it quick so you can iterate fast.


In [1]:
# 0) Environment & folders
import os, sys, math, json, random, time
from pathlib import Path
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi'] = 120

try:
    import torch
    TORCH_OK = True
except Exception as e:
    TORCH_OK = False
    print("PyTorch import failed:", e)

NOTEBOOK_CWD = Path.cwd().resolve()
CANDIDATES = [NOTEBOOK_CWD, NOTEBOOK_CWD.parent, NOTEBOOK_CWD.parent.parent]
BASE = None
for candidate in CANDIDATES:
    if (candidate / "src").is_dir():
        BASE = candidate
        break

if BASE is None:
    raise RuntimeError(f"Could not locate project root from {NOTEBOOK_CWD}")

print(f"Project root: {BASE}")
if str(BASE) not in sys.path:
    sys.path.insert(0, str(BASE))

for d in [
    'src', 'src/utils', 'src/models', 'src/baselines',
    'data', 'results', 'figures', 'figures/data_exploration',
    'figures/training_curves', 'figures/residual_heatmaps', 'figures/final_results']:
    (BASE / d).mkdir(parents=True, exist_ok=True)

DATA_DIR = BASE / 'data'
FIGURES_DIR = BASE / 'figures'
RESULTS_DIR = BASE / 'results'

print({
    'python': sys.version.split()[0],
    'numpy': np.__version__,
    'matplotlib': matplotlib.__version__,
    'torch': torch.__version__ if TORCH_OK else None,
})


Project root: /Users/amv10802/Documents/Neural-PDE-Option-Greeks
{'python': '3.11.7', 'numpy': '2.3.3', 'matplotlib': '3.10.7', 'torch': '2.8.0'}


In [2]:
# 1) Black–Scholes utilities — import if available, otherwise define minimal fallbacks
from math import log, sqrt, exp
from scipy.stats import norm


def _bs_price(S, K, T, t, sigma, r=0.05, option_type="call"):
    tau = max(T - t, 1e-6)
    d1 = (log(S/K) + (r + 0.5*sigma**2)*tau) / (sigma*sqrt(tau))
    d2 = d1 - sigma*sqrt(tau)
    if option_type == 'call':
        return S*norm.cdf(d1) - K*exp(-r*tau)*norm.cdf(d2)
    return K*exp(-r*tau)*norm.cdf(-d2) - S*norm.cdf(-d1)


def _bs_greeks(S, K, T, t, sigma, r=0.05):
    tau = max(T - t, 1e-6)
    d1 = (log(S/K) + (r + 0.5*sigma**2)*tau) / (sigma*sqrt(tau))
    d2 = d1 - sigma*sqrt(tau)
    delta = norm.cdf(d1)
    gamma = norm.pdf(d1)/(S*sigma*sqrt(tau))
    theta = -(S*norm.pdf(d1)*sigma)/(2*sqrt(tau)) - r*K*exp(-r*tau)*norm.cdf(d2)
    vega = S*norm.pdf(d1)*sqrt(tau)
    rho = K*tau*exp(-r*tau)*norm.cdf(d2)
    return dict(delta=delta, gamma=gamma, theta=theta, vega=vega, rho=rho)


try:
    from src.utils.black_scholes import bs_price, bs_greeks
    print("Imported bs_price/bs_greeks from src.utils.black_scholes")
except Exception:
    bs_price, bs_greeks = _bs_price, _bs_greeks
    print("Using fallback Black–Scholes implementations (define src/utils/black_scholes.py to override)")


Imported bs_price/bs_greeks from src.utils.black_scholes


In [3]:
# 2) Generate a tiny synthetic dataset (quick sanity only)
rng = np.random.default_rng(7)
K, T, r = 100.0, 2.0, 0.05
N_SMALL = 5_000
S = rng.uniform(20, 200, N_SMALL)
t = rng.uniform(0.01, 2.0, N_SMALL)
sigma = rng.uniform(0.05, 0.6, N_SMALL)
V = np.array([bs_price(S[i], K, T, t[i], sigma[i], r) for i in range(N_SMALL)], dtype=float)
small = np.stack([S, t, sigma, V], axis=1)
np.save(DATA_DIR / 'sanity_small.npy', small)
small.shape


(5000, 4)

In [4]:
# 3) Plot price and a couple of Greeks vs S (fixed t, sigma)
matplotlib.use('Agg')
S_line = np.linspace(20, 200, 400)
t0, sig0 = 1.0, 0.2
V_line = [bs_price(s, K, T, t0, sig0, r) for s in S_line]
G_line = [bs_greeks(s, K, T, t0, sig0, r) for s in S_line]
Delta = [g['delta'] for g in G_line]
Gamma = [g['gamma'] for g in G_line]

plt.figure(); plt.plot(S_line, V_line); plt.xlabel('S'); plt.ylabel('Price'); plt.title('BS Price vs S')
plt.tight_layout(); plt.savefig(FIGURES_DIR / 'data_exploration' / 'price_vs_S.png')

plt.figure(); plt.plot(S_line, Delta); plt.xlabel('S'); plt.ylabel('Delta'); plt.title('Delta vs S')
plt.tight_layout(); plt.savefig(FIGURES_DIR / 'data_exploration' / 'delta_vs_S.png')

plt.figure(); plt.plot(S_line, Gamma); plt.xlabel('S'); plt.ylabel('Gamma'); plt.title('Gamma vs S')
plt.tight_layout(); plt.savefig(FIGURES_DIR / 'data_exploration' / 'gamma_vs_S.png')

print('Saved: figures/data_exploration/*')


Saved: figures/data_exploration/*


In [5]:
# 4) Baselines: finite-difference & Monte Carlo vs analytic (single point + sweep)
def fd_delta_gamma(S0, eps=1e-2):
    V0 = bs_price(S0, K, T, 1.0, 0.2, r)
    Vp = bs_price(S0+eps, K, T, 1.0, 0.2, r)
    Vm = bs_price(S0-eps, K, T, 1.0, 0.2, r)
    delta = (Vp - Vm)/(2*eps)
    gamma = (Vp - 2*V0 + Vm)/(eps**2)
    return delta, gamma

def mc_pathwise_delta(S0, K=100, T=1.0, r=0.05, sigma=0.2, N=20_000, seed=0):
    rng = np.random.default_rng(seed)
    Z = rng.standard_normal(N)
    ST = S0*np.exp((r - 0.5*sigma**2)*T + sigma*np.sqrt(T)*Z)
    payoff = np.maximum(ST - K, 0)
    dS = ST / S0
    return np.exp(-r*T) * np.mean(dS * (ST > K))

S0 = 100
true = bs_greeks(S0, K, T, 1.0, 0.2, r)
fd_d, fd_g = fd_delta_gamma(S0)
mc_d = mc_pathwise_delta(S0)
print({
    'analytic_delta': true['delta'],
    'fd_delta': fd_d,
    'mc_delta': mc_d,
    'fd_gamma': fd_g,
    'analytic_gamma': true['gamma'],
})

{'analytic_delta': np.float64(0.6368306511756191), 'fd_delta': np.float64(0.6368306425763137), 'mc_delta': np.float64(0.6352971442332153), 'fd_gamma': np.float64(0.018762017077733617), 'analytic_gamma': np.float64(0.018762017345846895)}


In [6]:
# 5) Tiny PINN — quick training to confirm gradients & loss decrease
if not TORCH_OK:
    print('PyTorch not available — skipping PINN training cell.')
else:
    import torch, torch.nn as nn, torch.nn.functional as F
    from torch.utils.data import TensorDataset, DataLoader

    class ResidualBlock(nn.Module):
        def __init__(self, d=64):
            super().__init__()
            self.fc1, self.fc2 = nn.Linear(d,d), nn.Linear(d,d)
        def forward(self, x):
            h = F.relu(self.fc1(x))
            h = self.fc2(h)
            return F.relu(h + x)

    class TinyPINN(nn.Module):
        def __init__(self, d=64, L=3):
            super().__init__()
            self.inp = nn.Linear(3, d)
            self.blocks = nn.ModuleList([ResidualBlock(d) for _ in range(L)])
            self.out = nn.Linear(d, 1)
        def forward(self, x):
            x = F.relu(self.inp(x))
            for b in self.blocks:
                x = b(x)
            return self.out(x)

    arr = np.load(DATA_DIR / 'sanity_small.npy')
    S_t, t_t, s_t, V_t = [torch.tensor(arr[:,i], dtype=torch.float32) for i in range(4)]
    ds = TensorDataset(S_t, t_t, s_t, V_t)
    dl = DataLoader(ds, batch_size=1024, shuffle=True)

    def pde_residual(model, S, t, sigma, r=0.05):
        S = S.clone().requires_grad_(True)
        t = t.clone().requires_grad_(True)
        x = torch.stack([S, t, sigma], dim=1)
        V = model(x)
        ones = torch.ones_like(V)
        dVdS = torch.autograd.grad(V, S, grad_outputs=ones, create_graph=True)[0]
        d2VdS2 = torch.autograd.grad(dVdS, S, grad_outputs=torch.ones_like(dVdS), create_graph=True)[0]
        dVdt = torch.autograd.grad(V, t, grad_outputs=ones, create_graph=True)[0]
        return dVdt + 0.5*sigma**2*S**2*d2VdS2 + r*S*dVdS - r*V

    model = TinyPINN(d=64, L=3)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)

    losses = []
    EPOCHS = 5  # keep small for sanity
    for ep in range(EPOCHS):
        tot = 0.0
        for S_b, t_b, s_b, V_b in dl:
            opt.zero_grad()
            preds = model(torch.stack([S_b, t_b, s_b], dim=1))
            L_price = ((preds.squeeze() - V_b)**2).mean()
            L_pde = (pde_residual(model, S_b, t_b, s_b)**2).mean()
            loss = L_price + L_pde
            loss.backward(); opt.step()
            tot += float(loss.detach().cpu())
        losses.append(tot/len(dl))
        print(f"epoch {ep+1}/{EPOCHS}  loss={losses[-1]:.6f}")

    # plot loss
    plt.figure(); plt.plot(losses)
    plt.xlabel('epoch'); plt.ylabel('loss'); plt.title('Tiny PINN loss (sanity)')
    plt.tight_layout(); plt.savefig(FIGURES_DIR / 'training_curves' / 'tiny_pinn_loss.png')
    print('Saved: figures/training_curves/tiny_pinn_loss.png')


epoch 1/5  loss=2359.298193
epoch 2/5  loss=598.572986
epoch 3/5  loss=454.865417
epoch 4/5  loss=504.290186
epoch 5/5  loss=364.013098
Saved: figures/training_curves/tiny_pinn_loss.png


In [7]:
# 6) Surface & PDE residual heatmap (coarse)
if not TORCH_OK:
    print('PyTorch not available — skipping surface & residual plots.')
else:
    Sg = torch.linspace(20, 200, 50)
    sg = torch.linspace(0.05, 0.6, 50)
    Smesh, Sigmesh = torch.meshgrid(Sg, sg, indexing='ij')
    tgrid = torch.full_like(Smesh, 1.0)

    def pde_residual_grid(model, S, t, sigma, r=0.05):
        S = S.clone().requires_grad_(True)
        t = t.clone().requires_grad_(True)
        x = torch.stack([S.flatten(), t.flatten(), sigma.flatten()], dim=1)
        V = model(x).reshape_as(S)
        ones = torch.ones_like(V)
        dVdS = torch.autograd.grad(V, S, grad_outputs=ones, create_graph=True)[0]
        d2VdS2 = torch.autograd.grad(dVdS, S, grad_outputs=torch.ones_like(dVdS), create_graph=True)[0]
        dVdt = torch.autograd.grad(V, t, grad_outputs=ones, create_graph=True)[0]
        return (dVdt + 0.5*sigma**2*S**2*d2VdS2 + 0.05*S*dVdS - 0.05*V).detach()

    # Reuse TinyPINN from prior cell if present
    try:
        model
    except NameError:
        from math import isfinite
        print('TinyPINN not found in memory; re-instantiating untrained model for plotting.')
        import torch.nn as nn, torch.nn.functional as F
        class ResidualBlock(nn.Module):
            def __init__(self, d=64):
                super().__init__(); self.fc1, self.fc2 = nn.Linear(d,d), nn.Linear(d,d)
            def forward(self, x):
                h = F.relu(self.fc1(x)); h = self.fc2(h); return F.relu(h+x)
        class TinyPINN(nn.Module):
            def __init__(self, d=64, L=3):
                super().__init__(); self.inp = nn.Linear(3,d); self.blocks = nn.ModuleList([ResidualBlock(d) for _ in range(L)]); self.out = nn.Linear(d,1)
            def forward(self, x):
                x = F.relu(self.inp(x))
                for b in self.blocks: x = b(x)
                return self.out(x)
        model = TinyPINN()

    with torch.no_grad():
        Vpred = model(torch.stack([Smesh.flatten(), tgrid.flatten(), Sigmesh.flatten()], dim=1)).reshape_as(Smesh)

    plt.figure(figsize=(5,4))
    cp = plt.contourf(Sg.numpy(), sg.numpy(), Vpred.numpy().T, levels=30)
    plt.xlabel('S'); plt.ylabel('σ'); plt.title('Predicted Price Surface (coarse)')
    plt.colorbar(cp); plt.tight_layout(); plt.savefig(FIGURES_DIR / 'final_results' / 'surface_coarse.png')

    R = pde_residual_grid(model, Smesh, tgrid, Sigmesh)
    plt.figure(figsize=(5,4))
    cp = plt.contourf(Sg.numpy(), sg.numpy(), R.numpy().T, levels=30)
    plt.xlabel('S'); plt.ylabel('σ'); plt.title('PDE Residual (coarse, sanity)')
    plt.colorbar(cp); plt.tight_layout(); plt.savefig(FIGURES_DIR / 'residual_heatmaps' / 'pde_residual_coarse.png')
    print('Saved: figures/final_results/surface_coarse.png, figures/residual_heatmaps/pde_residual_coarse.png')


Saved: figures/final_results/surface_coarse.png, figures/residual_heatmaps/pde_residual_coarse.png


## ✅ What “good” looks like (for sanity)

- The three files in `figures/data_exploration/` exist and look reasonable (monotonic Delta, peaked Gamma).
- Baselines: FD Δ and MC Δ are in the same *ballpark* as analytic Δ at S≈K; FD Γ roughly matches analytic Γ.
- Tiny PINN: the loss plot decreases over a few epochs (it doesn’t need to be perfect here).
- Surface & PDE residual plots are produced (patterns may be rough without full training).

If any of these don’t happen, focus on that section first before moving to full experiments.